# Procesando coordenadas de los cortaderos

Procesando datos .kml que los obtenemos al exportar los datos de mymaps.

In [1]:
import json

from bs4 import BeautifulSoup
import numpy as np

In [2]:
def read4json(file_path):
    "Read configuration data form json file and return a list"
    with open(file_path, 'r') as data_file:
        data = json.load(data_file)
    return data

def save2json(file_path, data):
    "Save configuration data to a json file"
    with open(file_path, 'w') as data_file:
        json.dump(data, data_file, indent=4)

In [3]:
censo_path = '../crudos/censo2010cba.json'
kml_path = '../crudos/Ladrilleros.kml'
data_path = '../cortaderos_new.geojson'

In [4]:
# diccionario usados para obtener los valores del censo
# Extraido de:
# https://github.com/HackathonFOPEA/CensoCordoba2010/blob/master/index.html

acceso_a_gas = {
    'H14_01': 'Gas de red',
    'H14_02': 'Gas a granel',
    'H14_03': 'Gas en tubo',
    'H14_04': 'Gas en garrafa',
    'H14_05': 'Electricidad',
    'H14_06': 'Leña o carbón',
    'H14_07': 'Otro'
}

acceso_a_agua = {
    'H12_01': 'Red pública',
    'H12_02': 'Perforación con bomba a motor',
    'H12_03': 'Perforación con bomba manual',
    'H12_04': 'Pozo',
    'H12_05': 'Transporte por cisterna',
    'H12_06': 'Agua de lluvia, río, canal, arroyo o acequia'
}

tipo_vivienda = {
    'V08_01': 'Casa',
    'V08_02': 'Rancho',
    'V08_03': 'Casilla',
    'V08_04': 'Departamento',
    'V08_05': 'Pieza en inquilinato',
    'V08_06': 'Pieza en hotel familiar o pensión',
    'V08_07': 'Local no construido para habitación',
    'V08_08': 'Vivienda móvil',
    'V08_09': 'Persona/s viviendo en la calle',
}

area = {
    'V03_01': 'Vivienda area Urbano',
    'V03_02': 'Vivienda area Rural agrupado',
    'V03_03': 'Vivienda area Rural disperso'
}

banio = {
    'H01_01': 'Tiene baño o letrina',
    'H01_02': 'No tiene baño o letrina'
}

hacinamiento = {
    'H15_01': 'hacinamiento: Hasta   0.50 personas por cuarto',
    'H15_02': 'hacinamiento: 0.51  -  0.99  personas por cuarto',
    'H15_03': 'hacinamiento: 1.00  -  1.49  personas por cuarto',
    'H15_04': 'hacinamiento: 1.50  -  1.99  personas por cuarto',
    'H15_05': 'hacinamiento: 2.00  -  3.00  personas por cuarto',
    'H15_06': 'hacinamiento: Más de  3.00 personas por cuarto'
}

nivel_educativo = {
    'P03_01': 'Nivel educativo que cursa o cursó: Inicial (jardin, preescolar)',
    'P03_02': 'Nivel educativo que cursa o cursó: Primario',
    'P03_03': 'Nivel educativo que cursa o cursó: EGB',
    'P03_04': 'Nivel educativo que cursa o cursó: Secundario',
    'P03_05': 'Nivel educativo que cursa o cursó: Polimodal',
    'P03_06': 'Nivel educativo que cursa o cursó: Superior no universitario',
    'P03_07': 'Nivel educativo que cursa o cursó: Universitario',
    'P03_08': 'Nivel educativo que cursa o cursó: Post universitario',
    'P03_09': 'Nivel educativo que cursa o cursó: Educacion especial',
}

V03 = ['V03_01', 'V03_02', 'V03_03']
V08 = ['V08_01', 'V08_02', 'V08_03', 'V08_04', 'V08_05', 'V08_06', 'V08_07', 'V08_08', 'V08_09']
H01 = ['H01_01', 'H01_02']
H12 = ['H12_01', 'H12_02', 'H12_03', 'H12_04', 'H12_05', 'H12_06']
H14 = ['H14_01', 'H14_02', 'H14_03', 'H14_04', 'H14_05', 'H14_06', 'H14_07']
H15 = ['H15_01', 'H15_02', 'H15_03', 'H15_04', 'H15_05', 'H15_06']
P03 = ['P03_01', 'P03_02', 'P03_03', 'P03_04', 'P03_05', 'P03_06', 'P03_07', 'P03_08', 'P03_09']

In [5]:
data = open(kml_path).read()
bs = BeautifulSoup(data, "lxml")

In [6]:
bs.find('coordinates').text

'-63.9524943,-31.3270035,0.0'

In [7]:
censo = read4json(censo_path)
len_censo = len(censo[0]['features'])

In [8]:
todas_cortaderas = []

for coordenadas in bs.find_all('coordinates'):
    # Coordenadas de la cortadera
    lat, long, z = coordenadas.text.split(',')
    cortadera_pos = np.array([float(lat), float(long)])
    
    # calculo ladistancia de la cortadera a todos los puntos del censo
    pos = np.zeros((len_censo, 2))
    for i in range(len_censo):
        pos[i, 0] = censo[0]['features'][i]['properties']['coordinates'][0]
        pos[i, 1] = censo[0]['features'][i]['properties']['coordinates'][1]

    diff = (pos - cortadera_pos)**2
    suma = diff[:,0] + diff[:,1]
    minimo = suma.argmin()
    
    # obtengo los datos del agua del punto mas cercano del censo
    agua_dic = {}
    for key in H12:
        value = censo[0]['features'][minimo]['properties'][key]
        new_key = acceso_a_agua[key]
        agua_dic[new_key] = value

    # obtengo los datos del gas del punto mas cercano del censo
    gas_dic = {}    
    for key in H14:
        value = censo[0]['features'][minimo]['properties'][key]
        new_key = acceso_a_gas[key]
        gas_dic[new_key] = value
        
    # Obtengo los datos del tipo de vivienda
    vivienda_dic = {}
    for key in V08:
        value = censo[0]['features'][minimo]['properties'][key]
        new_key = tipo_vivienda[key]
        vivienda_dic[new_key] = value

    # Area
    area_dic = {}
    for key in V03:
        value = censo[0]['features'][minimo]['properties'][key]
        new_key = area[key]
        area_dic[new_key] = value

    # banio
    banio_dic = {}
    for key in H01:
        value = censo[0]['features'][minimo]['properties'][key]
        new_key = banio[key]
        banio_dic[new_key] = value

    # Hacinamiento
    hacinamiento_dic = {}
    for key in H15:
        value = censo[0]['features'][minimo]['properties'][key]
        new_key = hacinamiento[key]
        hacinamiento_dic[new_key] = value

    # Nivel educativo
    educacion_dic = {}
    for key in P03:
        value = censo[0]['features'][minimo]['properties'][key]
        new_key = nivel_educativo[key]
        educacion_dic[new_key] = value


    # densidad demografica
    densidad = censo[0]['features'][minimo]['properties']['P_TOTAL']
    
    # nuevo valor para el json de las cortaderas
    entry = {
            "geometry": {
                "coordinates": list(cortadera_pos),
                "type": "Point"
            },
            "type": "Feature",
            "properties": {
                "type": "cortaderos",
                "name": "Cortadero",
                "acceso_gas": gas_dic,
                "acceso_agua": agua_dic,
                "densidad": densidad,
                "vivienda": vivienda_dic,
                "area": area_dic,
                "banio": banio_dic,
                "hacinaiento": hacinamiento_dic,
                "educacion": educacion_dic
            }
        }

    todas_cortaderas.append(entry)

In [9]:
geojson = {
    "type": "FeatureCollection",
    "features": todas_cortaderas
    }

#save2json('../datos/cortaderos_new.geojson', geojson)
save2json('../cortaderos_new.geojson', geojson)